# Install packages

In [1]:
!pip install --upgrade transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 67.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [2]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 6.0 MB/s eta 0:00:00


# Code Generation

In [3]:
from transformers import AutoTokenizer
import transformers
import torch

code_tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-hf")
code_pipeline = transformers.pipeline(
    "text-generation",
    model="codellama/CodeLlama-7b-hf",
    torch_dtype=torch.float16,
    device_map="auto",
)




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [4]:
code_sequences = code_pipeline(
    'def selection_sort(',
    do_sample=True,
    temperature=0.2,
    top_p=0.9,
    num_return_sequences=1,
    eos_token_id=code_tokenizer.eos_token_id,
    max_length=200,
)
for s in code_sequences:
    print(f"Result: {s['generated_text']}")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Result: def selection_sort(self, arr):
        """
        Sorts the array using selection sort
        :param arr: Array to be sorted
        :return: Sorted array
        """
        for i in range(len(arr)):
            min_index = i
            for j in range(i + 1, len(arr)):
                if arr[j] < arr[min_index]:
                    min_index = j
            arr[i], arr[min_index] = arr[min_index], arr[i]
        return arr

    def insertion_sort(self, arr):
        """
        Sorts the array using insertion sort
        :param arr: Array to be sorted
        :return: Sorted array
        """
        for i in range(1, len(arr)):
            key = arr[i]
            j = i - 


# For auto filling

In [3]:
!pip install bitsandbytes

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

fill_model_id = "codellama/CodeLlama-7b-hf"
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
 )

fill_tokenizer = AutoTokenizer.from_pretrained(fill_model_id)
fill_model = AutoModelForCausalLM.from_pretrained(
    fill_model_id,
    quantization_config=quantization_config,
    #load_in_4bit=True,
    device_map="auto",
)





/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
fill_prompt = '''def check_palindrom(s: str) -> bool:
    """ <FILL_ME>
    return result
'''
inputs = fill_tokenizer(fill_prompt, return_tensors="pt").to("cuda")

inputs

{'input_ids': tensor([[    1, 32007,   822,  1423, 29918,  7830,   513,   456, 29898, 29879,
         29901,   851, 29897,  1599,  6120, 29901,    13,  1678,  9995, 29871,
         32008,    13,  1678,   736,  1121,    13, 32009]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]], device='cuda:0')}

In [6]:
output = fill_model.generate(
    inputs["input_ids"],
    max_new_tokens=400,
    do_sample=True,
    top_p=0.9,
    temperature=0.1,
)
output


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


tensor([[    1, 32007,   822,  1423, 29918,  7830,   513,   456, 29898, 29879,
         29901,   851, 29897,  1599,  6120, 29901,    13,  1678,  9995, 29871,
         32008,    13,  1678,   736,  1121,    13, 32009,  5596,   565,  1347,
           338,  5112,   513,   456,  9995,    13,  1678,  1121,   353,  5852,
            13,  1678,   363,   474,   297,  3464, 29898,  2435, 29898, 29879,
         22164,    13,  4706,   565,   269, 29961, 29875, 29962,  2804,   269,
         14352, 29896,   334,   474,   448, 29871, 29896,  5387,    13,  9651,
          1121,   353,  7700,    13,  9651,  2867, 32010,     2]],
       device='cuda:0')

In [7]:
output = output[0].to("cpu")
print(fill_tokenizer.decode(output))

<s> <PRE> def check_palindrom(s: str) -> bool:
    """  <SUF>
    return result
 <MID>Check if string is palindrom """
    result = True
    for i in range(len(s)):
        if s[i] != s[-1 * i - 1]:
            result = False
            break <EOT></s>


In [8]:
output

tensor([    1, 32007,   822,  1423, 29918,  7830,   513,   456, 29898, 29879,
        29901,   851, 29897,  1599,  6120, 29901,    13,  1678,  9995, 29871,
        32008,    13,  1678,   736,  1121,    13, 32009,  5596,   565,  1347,
          338,  5112,   513,   456,  9995,    13,  1678,  1121,   353,  5852,
           13,  1678,   363,   474,   297,  3464, 29898,  2435, 29898, 29879,
        22164,    13,  4706,   565,   269, 29961, 29875, 29962,  2804,   269,
        14352, 29896,   334,   474,   448, 29871, 29896,  5387,    13,  9651,
         1121,   353,  7700,    13,  9651,  2867, 32010,     2])

In [9]:
inputs['input_ids'].shape[1]

27

In [10]:
#output = output[0].to("cpu")

filling = fill_tokenizer.decode(output[inputs['input_ids'].shape[1]:], skip_special_tokens=True)
print(fill_prompt.replace("<FILL_ME>", filling))

def check_palindrom(s: str) -> bool:
    """ Check if string is palindrom """
    result = True
    for i in range(len(s)):
        if s[i] != s[-1 * i - 1]:
            result = False
            break
    return result

